# 读取数据集

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import jieba

In [2]:
df_train = pd.read_csv('./data/nCoV_100k_train.labled.csv',engine ='python')
df_test = pd.read_csv('./data/nCov_10k_test.csv',engine ='python')

In [3]:
print(df_train.head())
print(df_test.head())

               微博id        微博发布时间      发布人账号  \
0  4456072029125500  01月01日 23:50     存曦1988   
1  4456074167480980  01月01日 23:58   LunaKrys   
2  4456054253264520  01月01日 22:39  小王爷学辩论o_O   
3  4456061509126470  01月01日 23:08         芩鎟   
4  4455979322528190  01月01日 17:42   changlwj   

                                              微博中文内容  \
0  写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...   
1    开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?   
2  �偳癯空饩褪俏业�，爹，发烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，...   
3                     新年的第一天感冒又发烧的也太衰了但是我要想着明天一定会好的?   
4  问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...   

                                                微博图片 微博视频 情感倾向  
0  ['https://ww2.sinaimg.cn/orj360/005VnA1zly1gah...   []    0  
1                                                 []   []   -1  
2  ['https://ww2.sinaimg.cn/thumb150/006ymYXKgy1g...   []    1  
3  ['https://ww2.sinaimg.cn/orj360/005FL9LZgy1gah...   []    1  
4                                                

In [4]:
def cut_words(sentence):
    #print sentence
    return " ".join(jieba.cut(sentence))

In [5]:
MAX_SEQUENCE_LENGTH = 100
input_categories = '微博中文内容'
output_categories = '情感倾向'
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]

In [6]:
df_train['sentence_cuted'] =  df_train[input_categories].astype(str).apply(cut_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\37473\AppData\Local\Temp\jieba.cache
Loading model cost 0.692 seconds.
Prefix dict has been built successfully.


In [7]:
df_test['sentence_cuted'] =  df_test[input_categories].astype(str).apply(cut_words)

In [8]:
df_test.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,sentence_cuted
0,4456068992182160,01月01日 23:38,-精緻的豬豬女戰士-,#你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...,['https://ww2.sinaimg.cn/thumb150/745aa591ly1g...,[],# 你好 2020 # 新年 第一天 元气 满满的 早起 出门 买 早饭 结果 高估 了 自...
1,4456424178427250,01月02日 23:09,liujunyi88,大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...,[],[],大宝 又 感冒 鼻塞 咳嗽 了 ， 还有 发烧 。 队友 加班 几天 不回 。 感觉 自己 ...
2,4456797466940200,01月03日 23:53,ablsa,还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?,['https://ww3.sinaimg.cn/orj360/006fTidCly1gaj...,[],还要 去 输 两天 液 ， 这天 也 太 容易 感冒 发烧 了 ， 一定 要 多 喝 热水 啊 ?
3,4456791021108920,01月03日 23:27,喵吃鱼干Lynn,我太难了别人怎么发烧都没事就我一检查甲型流感?,[],[],我太难 了 别人 怎么 发烧 都 没事 就 我 一 检查 甲型 流感 ?
4,4457086404997440,01月04日 19:01,我的发小今年必脱单,果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...,[],[],果然 是 要 病 一场 的 喽 回来 第三天 开始 感冒 今儿 还 发烧 了 喉咙 眼睛 都...


In [9]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns].astype(int) + 1)
outputs = compute_output_arrays(df_train, output_categories)

In [10]:
outputs

array([1, 0, 2, ..., 1, 2, 1])

In [11]:
train = df_train
test = df_test

In [12]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, 
                                                  lower=False,filters="")# 对数据构造词典
tokenizer.fit_on_texts(train['sentence_cuted'].tolist()+test['sentence_cuted'].tolist())

In [13]:
train_ = tokenizer.texts_to_sequences(train['sentence_cuted'].values)# 文本 ---> 数字
test_ = tokenizer.texts_to_sequences(test['sentence_cuted'].values)# 同上

In [26]:
print(len(train_))
len(train_[0])

99913


92

# TFIDF构建文本特征

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),#(1,3)
            min_df=3,  # 4  5
            max_df=0.9, # 0.95 1.0 
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [27]:
train_term_doc = word_vec.fit_transform(train['sentence_cuted'])

In [28]:
test_term_doc = word_vec.transform(test['sentence_cuted'])

In [29]:
train_term_doc#sparse matrix 102277x2820641  np.array

<99913x196053 sparse matrix of type '<class 'numpy.float64'>'
	with 3559779 stored elements in Compressed Sparse Row format>

In [30]:
test_term_doc  #102277x2820641 稀疏矩阵可以直接在逻辑回归里面调用

<10000x196053 sparse matrix of type '<class 'numpy.float64'>'
	with 335787 stored elements in Compressed Sparse Row format>

# 逻辑回归构建模型

In [32]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=4, dual=False) # c 1.0  2 3 4 5  200w 10w
clf.fit(train_term_doc,outputs)#[0,18]


c:\tempbeforenewdisk\anaconda3\envs\jupyter\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
test_prob = clf.predict_proba(test_term_doc)

In [36]:
test_prob[0]# 4 +1 =5

array([0.49294859, 0.17924776, 0.32780364])

# 结果提交

In [37]:
test_pred = np.argmax(test_prob,axis=1)

In [38]:
test_pred

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [43]:
df_sub = df_test[['微博id']]

In [44]:
df_sub

,微博id
0,4456068992182160
1,4456424178427250
2,4456797466940200
3,4456791021108920
4,4457086404997440
...,...
9995,4464179518243680
9996,4464274073923100
9997,4464289160945130
9998,4465347950314820


In [46]:
df_sub['y'] = test_pred-1

c:\tempbeforenewdisk\anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
df_sub

,微博id,y
0,4456068992182160,-1
1,4456424178427250,0
2,4456797466940200,0
3,4456791021108920,-1
4,4457086404997440,-1
...,...,...
9995,4464179518243680,0
9996,4464274073923100,0
9997,4464289160945130,0
9998,4465347950314820,0


In [49]:
df_sub.columns=['id','y']

In [50]:
df_sub.head()

,id,y
0,4456068992182160,-1
1,4456424178427250,0
2,4456797466940200,0
3,4456791021108920,-1
4,4457086404997440,-1


In [51]:
df_sub.to_csv('test_sub.csv',index=False, encoding='utf-8')